In [28]:
import os, cv2
import numpy as np
import sys
from PIL import Image
from xml.etree.cElementTree import parse
# import dataset_utils.datasetUtils as datasetUtils
import time

#################################################
# import dataset_utils.datasetUtils as datasetUtils
#################################################
# If the script importing the module is in a package
# from .. import datasetUtils
#################################################
# If the script importing the module is not in a package
# import inspect

# current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parent_dir = os.path.dirname(current_dir)
# sys.path.insert(0, parent_dir)
import datasetUtils
#################################################
from pathos.multiprocessing import ProcessingPool as Pool


# from multiprocessing import Pool
#
# pool = Pool(processes=8)
def imageResize(imagePath, imageSize, bbox):
    image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
    if bbox != None:
        imageBbox = image[bbox[2]:bbox[3], bbox[0]:bbox[1], :]
        if len(imageBbox) == 0 or len(imageBbox[0]) == 0:
            imageResult = image
        else:
            imageResult = imageBbox
    else:
        imageResult = image
    imageResult = datasetUtils.imgAug(imageResult)
    imageResult = cv2.resize(imageResult, imageSize)
    return imageResult


class imagenetVidDataset(object):
    def __init__(self, dataPath, classNum=31, concurrentLength=4):
        self._classes = ['__background__',  # always index 0
                        'airplane', 'antelope', 'bear', 'bicycle',
                        'bird', 'bus', 'car', 'cattle',
                        'dog', 'domestic_cat', 'elephant', 'fox',
                        'giant_panda', 'hamster', 'horse', 'lion',
                        'lizard', 'monkey', 'motorcycle', 'rabbit',
                        'red_panda', 'sheep', 'snake', 'squirrel',
                        'tiger', 'train', 'turtle', 'watercraft',
                        'whale', 'zebra']
        self._classes_map = ['__background__',  # always index 0
                            'n02691156', 'n02419796', 'n02131653', 'n02834778',
                            'n01503061', 'n02924116', 'n02958343', 'n02402425',
                            'n02084071', 'n02121808', 'n02503517', 'n02118333',
                            'n02510455', 'n02342885', 'n02374451', 'n02129165',
                            'n01674464', 'n02484322', 'n03790512', 'n02324045',
                            'n02509815', 'n02411705', 'n01726692', 'n02355227',
                            'n02129604', 'n04468005', 'n01662784', 'n04530566',
                            'n02062744', 'n02391049']
        self._dataPath = dataPath
        self._classNum = classNum
        self._epoch = 0
        self._dataStart = 0
        self._dataLength = 0
        self._dataPointPathList = None
        self._classIdxConverter = None
        self._imageSize = (416, 416)
        self._concurrentLength = concurrentLength
        self._cellNum = (int(self._imageSize[0] / 32), int(self._imageSize[1] / 32))
        self._B = 2
        self._outputDim = self._B * 5 + self._classNum
        # self._loadDataPointPath2()
        self._loadDataPointPath()
        self._dataShuffle()

    def setImageSize(self, size=(416, 416)):
        self._imageSize = (size[0], size[1])

    def _loadDataPointPath(self):
        print 'load data point path...'
        self._dataPointPathList = []
        self._classIdxConverter = dict()
#         with open(self._dataPath + "/class.txt") as textFile:
#             lines = [line.split(" ") for line in textFile]
#         print lines

        trainPath = os.path.join(self._dataPath, 'Data')
        trainPath = os.path.join(trainPath, 'VID')
        trainPath = os.path.join(trainPath, 'train')
        subtrainPathList = os.listdir(trainPath)
        subtrainPathList.sort(key=datasetUtils.natural_keys)
        # print subtrainPathList
        subsubtrainPathList = []
        self._dataPointPathList = []
        for subtrainpath in subtrainPathList:
            sub_sub_train_path = os.listdir(os.path.join(trainPath, subtrainpath))
            sub_sub_train_path.sort(key=datasetUtils.natural_keys)
            subsubtrainPathList.append(sub_sub_train_path)
            for k in sub_sub_train_path:
                semi_finalPath = os.path.join(trainPath, subtrainpath, k)
                imgPathList = os.listdir(semi_finalPath)
                imgPathList.sort(key=datasetUtils.natural_keys)
                if len(imgPathList) < self._concurrentLength:
                    continue
                for img in range(len(imgPathList)-self._concurrentLength+1):
                    finalPath = os.path.join(semi_finalPath, imgPathList[img])
                    self._dataPointPathList.append(finalPath)
#         print self._dataPointPathList[-1]
#         print subsubtrainPathList[0]
#         print len(subsubtrainPathList)
#         print len(subsubtrainPathList[3])
        # print len(self._dataPointPathList)
        self._dataLength = len(self._dataPointPathList)
        print 'load done!'

    def _dataShuffle(self):
        # 'data list shuffle...'
        self._dataStart = 0
        np.random.shuffle(self._dataPointPathList)
        print "shuffle done!\n"
        # print len(self._dataPointPathList)
        # print self._dataPointPathList[0]

    def showImage(self, path, ind):
        index = str(ind).zfill(6)
        img_name = path + '/' + index + '.JPEG'

        # print img_name
        img = cv2.imread(img_name)
        cv2.imshow('image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def drawRect(self, img, box):
        img = cv2.rectangle(img, (box[1], box[3]), (box[0], box[2]), (255, 0, 0), 3)
        return img


    def getConcurrentImages(self, path, startind):
        seq_length = len(os.listdir(path))
        #if startind + self._concurrentLength -1 > seq_length -1:
        #    print 'Concurrent image frames should be inside the sequence!'
        #    return
        img_List = np.zeros([self._imageSize[0], self._imageSize[1], 3 * self._concurrentLength])

        output = np.zeros([self._cellNum[0], self._cellNum[1], self._outputDim], np.float32) ### 13 by 13 by 41

        # For loop for getting concurrent images
        for i in range(self._concurrentLength):
            index = str(startind + i).zfill(6)
            img_name = path + '/' + index + '.JPEG'

            img = cv2.imread(img_name, cv2.IMREAD_COLOR).astype('float')
#             print np.shape(img)

            # #### Draw Rectangles ####
            # img = cv2.imread(img_name)
            # img = self.drawRect(img, box_list_total[i][0])
            # cv2.imshow('image', img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

            # print ("after imread: {0:.6f}".format(time.time() - startTime))
            img = cv2.resize(img, self._imageSize)
#             print np.shape(img)
            #             img_List.append(img.copy())
            img_List[:,:,i * 3:(i+1) * 3] = img

            if i < self._concurrentLength - 1:
                continue

            ### Read xml for only last frame
            # print "k"
            xml_path = str.replace(img_name, 'Data', 'Annotations')
            xml_path = str.replace(xml_path, 'JPEG', 'xml')

            # print img_name
            # print xml_path

            startTime = time.time()
            xml_file = parse(xml_path)
            note = xml_file.getroot()
            obj_list_temp = note.findall('object')
            img_size = [float(note.find('size').findtext('width')), float(note.find('size').findtext('height'))]
            # print img_size

            obj_num = len(obj_list_temp)


            #### Read all Bounding Boxes on each frame ####
            # img = cv2.imread(img_name)

            for j in range(min(self._B, obj_num)):

                obj_temp = obj_list_temp[j]
                xmax = int(obj_temp.find('bndbox').findtext('xmax'))
                xmin = int(obj_temp.find('bndbox').findtext('xmin'))
                ymax = int(obj_temp.find('bndbox').findtext('ymax'))
                ymin = int(obj_temp.find('bndbox').findtext('ymin'))
                trackid = int(obj_temp.findtext('trackid'))
                obj_ind = int(self._classes_map.index(obj_temp.findtext('name')))
                # print obj_ind
                obj_class = self._classes[obj_ind]
                # print obj_class

                xc = (xmax + xmin) / 2.0
                yc = (ymax + ymin) / 2.0
                nx = int(xc / float(img_size[0] / self._cellNum[0]))
                ny = int(yc / float(img_size[1] / self._cellNum[1]))

                ### Draw Rect on every objects ###
                # img = self.drawRect(img, [xmax, xmin, ymax, ymin])

                output[ny, nx, j * 5 + 4] = 1
                box = np.array([xc / img_size[0], yc / img_size[1], (xmax - xmin) / img_size[0], (ymax - ymin) / img_size[1]], np.float32)
                output[ny, nx, j*5:(j+1)*5 - 1] = box
                output[ny, nx, self._B * 5 + obj_ind] = 1

            # print Obj_Indicator[:,:,1]
            # cv2.imshow('image', img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

        return np.array(img_List), output

    def newEpoch(self):
        self._epoch += 1
        self._dataStart = 0
        self._dataShuffle()

    def getNextBatch(self, batchSize=32):
        verystartTime = time.time()
        if self._dataStart + batchSize >= self._dataLength:
            self.newEpoch()
        dataStart = self._dataStart
        dataEnd = dataStart + batchSize
        self._dataStart = self._dataStart + batchSize

        # print "batch epoch is {:f}".format(self._epoch)

        # Getting Batch
        dataPathTemp = self._dataPointPathList[dataStart:dataEnd]
        # print dataPathTemp
        ImgBatchData = []
        OutBatchData = []
        # xml_path_list = []

        #         print ("after batch selection: {0:.6f}".format(time.time()-startTime))

        # Batch is composed of batchsize * concurrentLength number of image frames
        for path in dataPathTemp:
            box_list = []
            startTime = time.time()
            
            parent_path = os.path.abspath(os.path.join(path, '..')) # get the parent path
            ind = int(path.split('/')[-1].replace('.JPEG','')) # get the image index
            
#             print parent_path
#             print ind

            # then obtain concurrent images
            frame_list, LastFrame_Output = self.getConcurrentImages(parent_path, ind)
#             print frame_list
#             print LastFrame_Output

            ImgBatchData.append(frame_list)
            OutBatchData.append(LastFrame_Output)
        # print ("append: {0:.6f}".format(time.time()-startTime))

        ######## Display Image ########
        # self.showImage(path, ind)


        #         startTime = time.time()
        final_batchData = {
            'Paths': dataPathTemp,
            'Images': np.array(ImgBatchData),
            'Outputs': np.array(OutBatchData)
        }

        consumedTime = time.time() - verystartTime
        # print("consumed time: {0:.3f}".format(consumedTime))

        return final_batchData



################### test ##################

# data_path = '/hdd/data/ILSVRC/Data/VID'
# # data_path = '/ssdubuntu/data/ILSVRC/Data/VID'


# sample_img_path = '/ssdubuntu/data/ILSVRC/Data/VID/train/ILSVRC2015_VID_train_0000/ILSVRC2015_train_00000000'
# sample_xml_path = '/ssdubuntu/data/ILSVRC/Annotations/VID/train/ILSVRC2015_VID_train_0000/ILSVRC2015_train_00008006/000000.xml'
#
# # vid_data.showImage(sample_img_path, 0)

# # sample = vid_data.getNextBatch()
# # print(sample.get('Boxes'))
# # print(type(sample.get('Images')))
# # print(sample.get('Paths'))
# print np.shape(sample.get('Images'))
# print np.shape(sample.get('Outputs'))
# # vid_data.showImage(sample.get('Paths')[1], 0)


data_path = '/ssdubuntu/data/ILSVRC'
vid_data = imagenetVidDataset(data_path)
sample = vid_data.getNextBatch(8)
# print vid_data._dataPointPathList[0]
# #
boxes = sample.get('Outputs')
print np.shape(boxes)

first_box = boxes[...,:4]
# print first_box[0,:,:,0]
# images = sample.get('Images')
# print np.shape(images)

# print images[0, 0, 0, :]
first_conf = boxes[..., 4]
second_conf = boxes[..., 9]
conf = np.stack([first_conf, second_conf], -1)
print np.shape(conf)

ind = np.argwhere(first_conf==1)
print ind
ind_temp = np.concatenate([ind[0],[0]],-1)

print ind_temp

print boxes[ind_temp[0], ind_temp[1], ind_temp[2], ind_temp[3]]
ind2 = np.argwhere(second_conf==1)
print ind2
ind_final = np.argwhere(conf==1)
print ind_final
#
# print boxes[ind[0][0], ind[0][1], ind[0][2], 0:4]

load data point path...
load done!
shuffle done!

(8, 13, 13, 41)
(8, 13, 13, 2)
[[0 2 1]
 [1 7 2]
 [2 5 4]
 [3 7 9]
 [4 6 6]
 [5 6 5]
 [6 4 6]
 [7 6 5]]
[0 2 1 0]
0.091796875
[[ 0  2  7]
 [ 1  8 10]
 [ 3  6  4]]
[[ 0  2  1  0]
 [ 0  2  7  1]
 [ 1  7  2  0]
 [ 1  8 10  1]
 [ 2  5  4  0]
 [ 3  6  4  1]
 [ 3  7  9  0]
 [ 4  6  6  0]
 [ 5  6  5  0]
 [ 6  4  6  0]
 [ 7  6  5  0]]
